In [1]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

<ipython-input-1-863b54eaad45>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
odf = pd.read_csv(r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\bike_share_locations.csv")
odf.head()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination
0,14193596,19593,2156242,Knight Law Center,44.043053,-123.069570,2021-12-01,00:03,Origin
1,14193597,19710,2156242,Knight Law Center,44.043103,-123.069593,2021-12-01,00:04,Origin
2,14193598,19734,2024943,Eugene Train Station,44.055235,-123.092733,2021-12-01,00:06,Origin
3,14193600,19617,2203210,NaN,44.044918,-123.085150,2021-12-01,00:30,Origin
4,14193601,19779,2166326,Erb Memorial Union overflow,44.045250,-123.072697,2021-12-01,00:45,Origin


In [4]:
'Knight Library (Virtual Hub)' in odf.Location.unique()

True

In [5]:
odf[odf.Latitude.isnull() | odf.Longitude.isnull()]

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination


In [6]:
df = odf

In [7]:
df["geom"] = df["Latitude"].map(str) + ',' + df['Longitude'].map(str)
df["geom"][0]

'44.0430533333333,-123.06957'

In [8]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [9]:
tqdm_notebook.pandas()

In [10]:
df['address'] = df['geom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/8886 [00:00<?, ?it/s]

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,geom,address
0,14193596,19593,2156242,Knight Law Center,44.043053,-123.069570,2021-12-01,00:03,Origin,"44.0430533333333,-123.06957","(University of Oregon, Sunset Alley, Eugene, L..."
1,14193597,19710,2156242,Knight Law Center,44.043103,-123.069593,2021-12-01,00:04,Origin,"44.0431033333333,-123.069593333333","(University of Oregon, Sunset Alley, Eugene, L..."
2,14193598,19734,2024943,Eugene Train Station,44.055235,-123.092733,2021-12-01,00:06,Origin,"44.055235,-123.092733333333","(Peace Health Rides, Willamette Street, Eugene..."
3,14193600,19617,2203210,NaN,44.044918,-123.085150,2021-12-01,00:30,Origin,"44.0449183333333,-123.08515","(1351, Ferry Street, College Hill, Eugene, Lan..."
4,14193601,19779,2166326,Erb Memorial Union overflow,44.045250,-123.072697,2021-12-01,00:45,Origin,"44.04525,-123.072696666667","(University of Oregon, Sunset Alley, Eugene, L..."


In [11]:
df.address[1]

Location(University of Oregon, Sunset Alley, Eugene, Lane County, Oregon, 97403, United States, (44.044551549999994, -123.07173597767621, 0.0))

In [12]:
df.Location.isnull().values[3]

True

In [13]:
def getLoc(x):
    loc = x.raw['address']
    a = [key for key, value in loc.items()][0]
    b = [key for key, value in loc.items()][1]
    if  a == 'house_number':
        res = loc['road'] + ' ' + loc['house_number']
    else:
        res = loc[a]
        if res == "University of Oregon":
            res = "UO " + loc[b]
    return res

In [14]:
getLoc(df[df.Location.isnull().values].address.values[1])

'Millrace Studios'

In [15]:
df['Name'] = df.address.apply(lambda x: getLoc(x))

In [16]:
sorted(df.Name.unique())

['10th and Mill Building',
 '13th Avenue Market',
 '13th and Olive (Willamate)',
 '2125 Franklin',
 '5000s - Brooks Field',
 '5000s/6000s Brooks Field',
 '5th Street Public Market',
 '6th Street Alley',
 '7-Eleven',
 'Actors Cabaret of Eugene',
 'Adams Street 1108',
 'Adams Street 1116',
 'Adams Street 1122',
 'Addictive Behaviors',
 'Agate Street 1840',
 'Albertsons',
 'Alder Street 1848',
 'Alder Street 2300',
 'Alder Street 2301',
 'Alder Street 2398',
 'Alder Street 2788',
 'Alton Baker BMX Track',
 'Alton Baker Disc Golf Course',
 'Amazon Parkway',
 'Amazon Pool Facility',
 'Ambrosia Restaurant & Bar',
 'Anstett Hall',
 'Approaching UO Station',
 'Autzen Stadium',
 'Barnhart Hall',
 'Best Western Hayward Inn',
 'Bethel',
 'Bicycle Way of Life',
 "Bill & Tim's Barbeque",
 'Black Cultural Center',
 'Blair Alley Barcade',
 'Bloom Hair Studio',
 'Boone Insurance Associates',
 'Bristol Street 2121',
 'Broadway Connector',
 "Brun's Apple Market",
 'Burrito Boy',
 'Cafe Coffee Day',
 'Ca

In [17]:
for n in df.loc[df.Location.isnull(),'Name'].unique():
    if all(df.loc[df['Name'] == n, 'Location'].isnull()):
        df.loc[df['Name'] == n, 'Location'] = n
    else:
        v = df.loc[df['Name'] == n, 'Location']
        df.loc[((df.Location.isnull()) & df['Name'] == n), 'Location'] = v[v.notnull()].values[0]    

In [19]:
df['Location'] = df['Location'].str.replace('University of Oregon','UO')

In [22]:
def remove_street_number(x):
    x = str(x)
    if len(x.split(' ')[-2:-1]) == 0:
        res = x
    elif (x.split(' ')[-2:-1][0] in ['Avenue', 'Street', 'Alley']) and (x.split(' ')[-1:][0].isdigit()):
        res = x.rsplit(' ', 1)[0]
    else:
        res = x
    
    return res

In [23]:
df['Location'] = df['Location'].apply(lambda x: remove_street_number(x))

In [24]:
df['Location'] = df['Location'].str.replace(' -',',')

In [34]:
df.loc[df.Location == 'nan', 'Name']

6       13th and Olive (Willamate)
19      13th and Olive (Willamate)
58                 UO Sunset Alley
112                UO Sunset Alley
125                UO Sunset Alley
                   ...            
8784                Florence Apt's
8836                Kalapuya Illhi
8856        10th and Mill Building
8876    13th and Olive (Willamate)
8880                  FedEx Office
Name: Name, Length: 499, dtype: object

In [35]:
df.loc[df.Location == 'nan', 'Location'] = df.loc[df.Location == 'nan', 'Name']

In [36]:
for i in df.RouteID.unique():
    idx1 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Origin'))
    idx2 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Destination'))
    pathID = df.loc[idx1, 'Location'].values[0] + ' - ' + df.loc[idx2, 'Location'].values[0]
    df.loc[idx1, 'PathID'] = pathID
    df.loc[idx2, 'PathID'] = pathID

In [26]:
df.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,geom,address,Name,PathID
8881,14209403,19712,2211448,17th & Hilyard,44.041062,-123.081933,2021-12-31,22:12,Destination,"44.0410616666667,-123.081933333333","(Peace Lutheran, East 17th Avenue, College Hil...",Peace Lutheran,"17th & Pearl, West side - 17th & Hilyard"
8882,14209412,19763,1733742,Glenwood,44.045285,-123.080093,2021-12-31,23:03,Destination,"44.045285,-123.080093333333","(Glenwood, 1340, Alder Street, Eugene, Lane Co...",Glenwood,Monroe St & Blair Blvd - Glenwood
8883,14209416,19631,1354450,Maude Kerns Art Center,44.042795,-123.064628,2021-12-31,23:27,Destination,"44.042795,-123.064628333333","(Maude Kerns Art Center, Orchard Alley, Eugene...",Maude Kerns Art Center,13th & Kincaid - Maude Kerns Art Center
8884,14209417,19755,1354450,Villard Street,44.042743,-123.064662,2021-12-31,23:27,Destination,"44.0427433333333,-123.064661666667","(1544, Villard Street, Eugene, Lane County, Or...",Villard Street 1544,13th & Kincaid - Villard Street
8885,14209418,19628,1354450,Villard Street,44.042865,-123.064647,2021-12-31,23:28,Destination,"44.042865,-123.064646666667","(1532, Villard Street, Eugene, Lane County, Or...",Villard Street 1532,13th & Kincaid - Villard Street


In [37]:
ndf = df.drop(['geom', 'address'], axis=1)

In [31]:
ndf.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,Name,PathID
8881,14209403,19712,2211448,17th & Hilyard,44.041062,-123.081933,2021-12-31,22:12,Destination,Peace Lutheran,"17th & Pearl, West side - 17th & Hilyard"
8882,14209412,19763,1733742,Glenwood,44.045285,-123.080093,2021-12-31,23:03,Destination,Glenwood,Monroe St & Blair Blvd - Glenwood
8883,14209416,19631,1354450,Maude Kerns Art Center,44.042795,-123.064628,2021-12-31,23:27,Destination,Maude Kerns Art Center,13th & Kincaid - Maude Kerns Art Center
8884,14209417,19755,1354450,Villard Street,44.042743,-123.064662,2021-12-31,23:27,Destination,Villard Street 1544,13th & Kincaid - Villard Street
8885,14209418,19628,1354450,Villard Street,44.042865,-123.064647,2021-12-31,23:28,Destination,Villard Street 1532,13th & Kincaid - Villard Street


In [38]:
ndf['Name'] = ndf['Name'].str.replace('University of Oregon','UO')
ndf['Name'] = ndf['Name'].str.replace('U of O','UO')

In [39]:
ndf.to_csv(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\pathID.csv', index=False)